# 8.4 BERT를 이용한 리뷰 문장에 대한 감정 분석 모델 구현 및 학습, 추론

BERT를 사용하여 IMDb 데이터의 포지티브/네거티브 분류 모델을 학습시켜, 추론합니다. 또한 추론시에 Self-Attention을 시각화합니다.


※ 이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.

# 8.4 학습 목표

1.	BERT의 vocabulary를 torchtext에서 사용하는 구현 방법을 이해한다
2.	BERT에 분류 작업용의 어댑터 모듈을 추가하고, 감정 분석을 실시하는 모델을 구현할 수 있다
3.	BERT를 파인 튜닝하여, 모델을 학습할 수 있다
4.  BERT의 Self-Attention 가중치를 시각화하고, 추론의 설명을 시도할 수 있다


# 사전 준비

- 도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다

In [1]:
import random
import time
import numpy as np
from tqdm import tqdm
import torch 
from torch import nn
import torch.optim as optim
import torchtext


In [2]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

# IMDb 데이터를 읽어들여, DataLoader 작성(BERT의 Tokenizer 사용)

In [3]:
# 전처리 및 단어 분할을 묶은 함수 작성
import re
import string
from utils.bert import BertTokenizer
# "utils" 폴더의 bert.py를 불러들임


def preprocessing_text(text):
    '''IMDb의 전처리'''
    # 개행 코드 삭제
    text = re.sub('<br />', '', text)

    # 쉼표, 마침표 이외의 기호를 공백(스페이스)으로 대체
    for p in string.punctuation:
        if (p == ".") or (p == ","):
            continue
        else:
            text = text.replace(p, " ")

    # 마침표 등의 전후에 공백을 넣음
    text = text.replace(".", " . ")
    text = text.replace(",", " , ")
    return text


# 단어 분할용 Tokenizer 준비
tokenizer_bert = BertTokenizer(
    vocab_file="./vocab/bert-base-uncased-vocab.txt", do_lower_case=True)


# 전처리와 단어 분할을 묶은 함수를 정의
# 단어 분할 함수를 전달하므로, tokenizer_bert 대신 tokenizer_bert.tokenize를 전달하는 점에 주의
def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)  # tokenizer_bert
    return ret


In [4]:
# 데이터를 읽었을 때, 내용에 대해 수행할 처리를 정의합니다
max_length = 256

TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True, fix_length=max_length, init_token="[CLS]", eos_token="[SEP]", pad_token='[PAD]', unk_token='[UNK]')
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# (주석) 각 인수를 재확인
# sequential: 데이터의 길이가 가변인가? 문장은 길이가 다양하므로 True. 라벨은 False
# tokenize: 문장을 읽을 때, 전처리 및 단어 분할 함수를 정의
# use_vocab: 단어를 vocabulary에 추가할지 여부
# lower: 알파벳이 존재할 때 소문자로 변환할지 여부
# include_length: 문장의 단어 수 데이터를 포함할지 여부
# batch_first: 미니 배치 차원을 선두에 제공할지 여부
# fix_length: 전체 문장을 지정한 길이가 되도록 padding
# init_token, eos_token, pad_token, unk_token: 문장 선두, 문장 말미, padding, 미지어에 대해, 어떤 단어를 부여하는지 지정


In [5]:
# "data" 폴더에서 각 tsv 파일을 읽습니다.
# BERT용으로 처리하므로, 10분 정도 시간이 걸립니다
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='IMDb_train.tsv',
    test='IMDb_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])

# torchtext.data.Dataset의 split 함수로 훈련 데이터와 validation 데이터를 나눈다
train_ds, val_ds = train_val_ds.split(
    split_ratio=0.8, random_state=random.seed(1234))


In [6]:
# BERT는 BERT가 가진 모든 단어로 BertEmbedding 모듈을 작성하므로, vocabulary는 전체 단어를 사용합니다
# 따라서 훈련 데이터로 vocabulary를 만들지 않습니다

# 우선 BERT용의 단어 사전을 사전형 변수에 준비합니다
from utils.bert import BertTokenizer, load_vocab

vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file="./vocab/bert-base-uncased-vocab.txt")


# 이대로, TEXT.vocab.stoi= vocab_bert (stoi는 string_to_ID로, 단어에서 ID로의 사전)로 하고 싶지만, 
# 일단 bulild_vocab를 실행하지 않으면 TEXT 오브젝트가 vocab의 멤버 변수를 갖지 않는다.
# ('Field' object has no attribute 'vocab'라는 에러가 발생합니다)

# 한 번 적당히 build_vocab에서 vocabulary를 작성하고, BERT의 vocabulary를 덮어씁니다
TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert


In [7]:
# DataLoader 작성(torchtext에서, 단순히 iterater라고 불립니다)
batch_size = 32  # BERT에서는 16, 32 근처를 사용

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

# 사전 객체로 정리
dataloaders_dict = {"train": train_dl, "val": val_dl}


In [8]:
# 동작 확인 - 검증 데이터 세트로 확인
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[ 101, 2023, 3185,  ...,    0,    0,    0],
        [ 101, 2043, 1045,  ...,    0,    0,    0],
        [ 101, 1045, 2066,  ...,    0,    0,    0],
        ...,
        [ 101, 1045, 2876,  ..., 1997, 1996,  102],
        [ 101, 2023, 2003,  ...,    0,    0,    0],
        [ 101, 1045, 2293,  ...,    0,    0,    0]]), tensor([128, 158, 178, 150, 112, 142, 248,  90, 256, 176, 224, 250, 256, 142,
        256, 200, 199, 185, 256, 256, 122, 184,  97, 148, 161, 211, 256, 256,
         57, 256, 146, 129]))
tensor([1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 0, 1, 1, 1])


In [9]:
# 미니 배치의 첫번째 문장을 확인
text_minibatch_1 = (batch.Text[0][1]).numpy()

# ID를 단어로 되돌림
text = tokenizer_bert.convert_ids_to_tokens(text_minibatch_1)

print(text)


['[CLS]', 'when', 'i', 'saw', 'this', 'movie', ',', 'i', 'was', 'amazed', 'that', 'it', 'was', 'only', 'a', 'tv', 'movie', '.', 'i', 'think', 'this', 'movie', 'should', 'have', 'been', 'in', 'theaters', '.', 'i', 'have', 'seen', 'many', 'movies', 'that', 'are', 'about', 'rape', ',', 'but', 'this', 'one', 'stands', 'out', '.', 'this', 'movie', 'has', 'a', 'kind', 'of', 'realism', 'that', 'is', 'very', 'rarely', 'found', 'in', 'movies', 'today', ',', 'let', 'alone', 'tv', 'movies', '.', 'it', 'tells', 'a', 'story', 'that', 'i', 'm', 'sure', 'is', 'very', 'realistic', 'to', 'many', 'rape', 'victims', 'in', 'small', 'towns', 'today', ',', 'and', 'i', 'found', 'it', 'to', 'be', 'very', 'bel', '##ie', '##vable', 'which', 'is', 'something', 'hard', 'to', 'find', 'in', 'other', 'rape', 'centered', 'movies', '.', 'i', 'also', 'thought', 'that', 'ti', '##ffa', '##ni', 'the', '##isse', '##n', 'and', 'brian', 'austin', 'green', 'were', 'awesome', 'in', 'the', 'parts', 'that', 'they', 'played', '.'

# 감정 분석용 BERT 모델 구축

In [10]:
from utils.bert import get_config, BertModel, set_learned_params

# 모델 설정 JSON 파일을 오브젝트 변수로 가져옵니다
config = get_config(file_path="./weights/bert_config.json")

# BERT 모델을 작성합니다
net_bert = BertModel(config)

# BERT 모델에 학습된 파라미터를 설정합니다
net_bert = set_learned_params(
    net_bert, weights_path="./weights/pytorch_model.bin")


bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.beta→embeddings.LayerNorm.beta
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.output

In [11]:
class BertForIMDb(nn.Module):
    '''BERT 모델에 IMDb의 포지티브/네거티브 판정 부분을 연결한 모델'''

    def __init__(self, net_bert):
        super(BertForIMDb, self).__init__()

        # BERT 모듈
        self.bert = net_bert  # BERT 모델

        # head에 포지티브/네거티브 예측을 추가
        # 입력은 BERT의 출력 특징량의 차원, 출력은 포지티브/네거티브의 두 가지
        self.cls = nn.Linear(in_features=768, out_features=2)

        # 가중치 초기화 처리
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, output_all_encoded_layers=False, attention_show_flg=False):
        '''
        input_ids:  [batch_size, sequence_length] 문장의 단어 ID를 나열
        token_type_ids:  [batch_size, sequence_length] 각 단어가 1번째 문장인지, 2번째 문장인지를 나타내는 id
        attention_mask: Transformer의 마스크와 같은 기능의 마스킹
        output_all_encoded_layers: 반환 값을 전체 TransformerBlock 모듈의 출력으로 할지, 
        최후 층만으로 한정할지의 플래그.
        attention_show_flg: Self-Attention의 가중치를 반환할지의 플래그
        '''

        # BERT의 기본 모델 부분의 순전파
        # 순전파한다
        if attention_show_flg == True:
            '''attention_show의 경우, attention_probs도 반환한다'''
            encoded_layers, pooled_output, attention_probs = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)
        elif attention_show_flg == False:
            encoded_layers, pooled_output = self.bert(
                input_ids, token_type_ids, attention_mask, output_all_encoded_layers, attention_show_flg)

        # 입력 문장의 첫단어 번째 [CLS]의 특징량을 사용하여, 포지티브/네거티브를 분류합니다
        vec_0 = encoded_layers[:, 0, :]
        vec_0 = vec_0.view(-1, 768)  # size를 [batch_size, hidden_size로 변환
        out = self.cls(vec_0)

        # attention_show의 경우, attention_probs(최후)도 반환합니다
        if attention_show_flg == True:
            return out, attention_probs
        elif attention_show_flg == False:
            return out


In [12]:
# 모델 구축
net = BertForIMDb(net_bert)

# 훈련 모드 설정
net.train()

print('네트워크 설정 완료')


네트워크 설정 완료


# BERT의 파인 튜닝을 위한 설정

In [13]:
# 기울기 계산을 최후의 BertLayer 모듈과 추가한 분류 어댑터만 실행

# 1. 먼저 전부를, 기울기 계산 False로 한다
for name, param in net.named_parameters():
    param.requires_grad = False

# 2. 최후의 BertLayer 모듈을 기울기 계산하도록 변경
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

# 3. 식별기를 기울기를 계산하도록 변경
for name, param in net.cls.named_parameters():
    param.requires_grad = True


In [14]:
# 최적화 기법 설정

# BERT의 원래 부분을 파인 튜닝
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

# 손실함수 설정
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()를 계산한 후 nn.NLLLoss(negative log likelihood loss)를 계산


# 학습 및 검증 실시

In [15]:
# 모델을 학습시키는 함수를 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    print('-----start-------')

    # 네트워크를 GPU로
    net.to(device)

    # 네트워크가 어느 정도 고정되면, 고속화시킨다
    torch.backends.cudnn.benchmark = True

    # 미니 배치 크기
    batch_size = dataloaders_dict["train"].batch_size

    # epoch 루프
    for epoch in range(num_epochs):
        # epoch별의 훈련 및 검증 루프
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # 모델을 훈련 모드로
            else:
                net.eval()   # 모델을 검증모드로

            epoch_loss = 0.0  # epoch의 손실합
            epoch_corrects = 0  # epoch의 정답수
            iteration = 1

            # 개시 시간을 저장
            t_epoch_start = time.time()
            t_iter_start = time.time()

            # 데이터 로더에서 미니 배치를 꺼내는 루프
            for batch in (dataloaders_dict[phase]):
                # batch는 Text와 Lable의 사전형 변수

                # GPU가 사용 가능하면 GPU로 데이터를 보낸다
                inputs = batch.Text[0].to(device)  # 문장
                labels = batch.Label.to(device)  # 라벨

                # optimizer 초기화
                optimizer.zero_grad()

                # 순전파(forward) 계산
                with torch.set_grad_enabled(phase == 'train'):

                    # BertForIMDb에 입력
                    outputs = net(inputs, token_type_ids=None, attention_mask=None,
                                  output_all_encoded_layers=False, attention_show_flg=False)

                    loss = criterion(outputs, labels)  # 손실 계산

                    _, preds = torch.max(outputs, 1)  # 라벨 예측

                    # 훈련시에는 역전파
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iter에 1번, loss를 표시
                            t_iter_finish = time.time()
                            duration = t_iter_finish - t_iter_start
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('반복 {} || Loss: {:.4f} || 10iter: {:.4f} sec. || 이 반복의 정답률: {}'.format(
                                iteration, loss.item(), duration, acc))
                            t_iter_start = time.time()

                    iteration += 1

                    # 손실과 정답 수의 합계를 갱신
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epoch별의 loss과 정답률
            t_epoch_finish = time.time()
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
            t_epoch_start = time.time()

    return net


In [16]:
# 학습 및 검증을 실행한다. 1epoch에 20분 정도 걸립니다.
num_epochs = 2
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


사용 장치:  cuda:0
-----start-------
반복 10 || Loss: 0.7256 || 10iter: 11.7782 sec. || 이 반복의 정답률: 0.53125
반복 20 || Loss: 0.6693 || 10iter: 11.0131 sec. || 이 반복의 정답률: 0.59375
반복 30 || Loss: 0.6792 || 10iter: 11.0456 sec. || 이 반복의 정답률: 0.5625
반복 40 || Loss: 0.6383 || 10iter: 11.1244 sec. || 이 반복의 정답률: 0.625
반복 50 || Loss: 0.6787 || 10iter: 11.0196 sec. || 이 반복의 정답률: 0.5625
반복 60 || Loss: 0.6298 || 10iter: 11.1094 sec. || 이 반복의 정답률: 0.625
반복 70 || Loss: 0.5353 || 10iter: 11.1385 sec. || 이 반복의 정답률: 0.6875
반복 80 || Loss: 0.6848 || 10iter: 11.1681 sec. || 이 반복의 정답률: 0.6875
반복 90 || Loss: 0.4199 || 10iter: 11.0776 sec. || 이 반복의 정답률: 0.78125
반복 100 || Loss: 0.3817 || 10iter: 11.1341 sec. || 이 반복의 정답률: 0.875
반복 110 || Loss: 0.4331 || 10iter: 11.1551 sec. || 이 반복의 정답률: 0.8125
반복 120 || Loss: 0.5997 || 10iter: 11.1967 sec. || 이 반복의 정답률: 0.65625
반복 130 || Loss: 0.3853 || 10iter: 11.1050 sec. || 이 반복의 정답률: 0.75
반복 140 || Loss: 0.2892 || 10iter: 11.1738 sec. || 이 반복의 정답률: 0.90625
반복 150 || Loss: 0.2563 |

In [17]:
# 학습한 네트워크 파라미터를 저장합니다
save_path = './weights/bert_fine_tuning_IMDb.pth'
torch.save(net_trained.state_dict(), save_path)


In [18]:
# 테스트 데이터의 정답률을 구한다
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # 모델을 검증모드로
net_trained.to(device)  #  GPU가 사용 가능하면 GPU에 보낸다

# epoch의 정답수를 기록하는 변수
epoch_corrects = 0

for batch in tqdm(test_dl):  # test 데이터의 DataLoader
    # batch는 Text와 Lable의 사전 오브젝트
    # GPU가 사용 가능하면 GPU로 데이터를 보낸다
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 문장
    labels = batch.Label.to(device)  # 라벨

    # 순전파(forward) 계산
    with torch.set_grad_enabled(False):

        # BertForIMDb에 입력
        outputs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                              output_all_encoded_layers=False, attention_show_flg=False)

        loss = criterion(outputs, labels)  # 손실 계산
        _, preds = torch.max(outputs, 1)  # 라벨을 예측
        epoch_corrects += torch.sum(preds == labels.data)  # 정답수의 합계를 갱신

# 정답률
epoch_acc = epoch_corrects.double() / len(test_dl.dataset)

print('테스트 데이터 {}개에서 정답률: {:.4f}'.format(len(test_dl.dataset), epoch_acc))


100%|██████████| 782/782 [13:41<00:00,  1.05s/it]


테스트 데이터 25000개에서 정답률: 0.9038


# Attention의 시각화

In [19]:
# batch_size를 64으로 한 테스트 데이터로 DataLoader를 작성
batch_size = 64
test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)


In [20]:
# BertForIMDb로 처리

# 미니 배치의 준비
batch = next(iter(test_dl))

# GPU가 사용 가능하면 GPU로 데이터를 보낸다
inputs = batch.Text[0].to(device)  # 문장
labels = batch.Label.to(device)  # 라벨

outputs, attention_probs = net_trained(inputs, token_type_ids=None, attention_mask=None,
                                       output_all_encoded_layers=False, attention_show_flg=True)

_, preds = torch.max(outputs, 1)  # 라벨을 예측


In [21]:
# HTML 작성 함수 구현
def highlight(word, attn):
    "Attention 값이 크면 문자 배경을 진한 빨간색으로 하는 html을 출력하는 함수"

    html_color = '#%02X%02X%02X' % (
        255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}"> {}</span>'.format(html_color, word)


def mk_html(index, batch, preds, normlized_weights, TEXT):
    "HTML 데이터를 작성한다"

    # index의 결과를 추출
    sentence = batch.Text[0][index]  # 문장
    label = batch.Label[index]  # 라벨
    pred = preds[index]  # 예측

    # 라벨과 예측 결과를 문자로 대체
    if label == 0:
        label_str = "Negative"
    else:
        label_str = "Positive"

    if pred == 0:
        pred_str = "Negative"
    else:
        pred_str = "Positive"

    # 표시용의 HTML을 작성한다
    html = '정답 라벨: {}<br>추론 라벨: {}<br><br>'.format(label_str, pred_str)

    # Self-Attention의 가중치를 가시화. Multi-Head가 12개이므로, 12종류의 attention이 존재
    for i in range(12):

        # index의 Attention을 추출하고 규격화
        # 0번째 단어 [CLS]의, i번째의 Multi-Head Attention를 꺼내
        # index는 미니 배치의 몇 번째 데이터인지를 나타냄
        attens = normlized_weights[index, i, 0, :]
        attens /= attens.max()

        html += '[BERT의 Attention을 시각화_' + str(i+1) + ']<br>'
        for word, attn in zip(sentence, attens):

            # 단어가 [SEP]인 경우 문장의 끝이므로 break
            if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
                break

            # 함수 highlight로 색을 칠하고, 함수 tokenizer_bert.convert_ids_to_tokens로 ID를 단어로 되돌림
            html += highlight(tokenizer_bert.convert_ids_to_tokens(
                [word.numpy().tolist()])[0], attn)
        html += "<br><br>"

    # 12종류의 Attention의 평균을 구한다. 최대치로 규격화
    all_attens = attens*0  # all_attens이라는 변수를 작성한다
    for i in range(12):
        attens += normlized_weights[index, i, 0, :]
    attens /= attens.max()

    html += '[BERT의 Attention을 시각화_ALL]<br>'
    for word, attn in zip(sentence, attens):

        # 단어가 [SEP]일 경우는 문장의 끝이므로 break
        if tokenizer_bert.convert_ids_to_tokens([word.numpy().tolist()])[0] == "[SEP]":
            break

        # 함수 highlight로 색을 칠하고, 함수 tokenizer_bert.convert_ids_to_tokens로 ID를 단어로 되돌림
        html += highlight(tokenizer_bert.convert_ids_to_tokens(
            [word.numpy().tolist()])[0], attn)
    html += "<br><br>"

    return html


In [22]:
from IPython.display import HTML

index = 3  # 출력할 데이터
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력


In [23]:
index = 61  # 출력할 데이터
html_output = mk_html(index, batch, preds, attention_probs, TEXT)  # HTML 작성
HTML(html_output)  # HTML 형식으로 출력


끝